In [12]:
### Import libraries
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

# import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

import time

In [34]:
### define KerasDemo02 class

class KerasDemo02:
    '''
        Loads NIST data and create a model to predict the digit
    '''
    def __init__(self):
        # The digits dataset
        self._digits = load_digits()
      
    def printDataDescription(self):
        print(self._digits.DESCR)

    def plotRandomDigit(self):
        id = np.random.randint(len(self._digits.target))
        self.plotDigit(id)

    def plotDigit(self,index):
        if(index > len(self._digits.target)-1):
            print(f'Index out of bound of data. Data size:{len(self._digits.target)-1}')
        else:
            plt.figure(figsize = (2, 2))
            plt.imshow(self._digits.images[index], cmap = 'gray')
            plt.xticks([])
            plt.yticks([])
            plt.show()
    
    def prepareDataForMode(self):
        # Prepare input data
        # input
        self._X = self._digits.data.astype(np.uint8)
        self._n_cols = self._X.shape[1]

        # output
        self._target = self._digits.target.astype(np.uint8)

        # Convert the target to categorical
        self._y = to_categorical(
            self._target,
            num_classes = len(set(self._target)),
            dtype = 'uint8')

    def splitTrainingAndTestData(self):
        self._X_train, self._X_test, self._y_train, self._y_test = train_test_split(
        self._X, self._y, 
        test_size = 0.3,
        random_state = 65)
    
    def createModel(self):
        # Set up the model architecture
        self._model = Sequential()
        # Add the first hidden layer
        self._model.add(Dense(100, activation = 'relu', input_shape = (self._n_cols, )))
        # Add the second hidden layer
        self._model.add(Dense(50, activation = 'relu'))
        # Add the output layer
        self._model.add(Dense(10, activation = 'softmax'))

    def compileModel(self):
        # Compile the model
        self._model.compile(
            optimizer = 'sgd',
            loss = 'categorical_crossentropy',
            metrics = ['accuracy'])


    def printModelSummary(self):
        print(self._model.summary())

    def trainModel(self, verboseSetting=0, epochNum=100, batch_size=20):
        # Fit the model
        print(f"Fitting the model in {epochNum} epochs")
        start_time = time.time()
        self._history = self._model.fit(
            self._X_train,
            self._y_train,
            validation_split = 0.25,
            batch_size = batch_size,
            epochs = epochNum,
            verbose = verboseSetting)
        print(f"Model has been trained")
        score = self._model.evaluate(self._X_train, self._y_train, batch_size = 315)
        print('\nTraining loss: %.6f, Training accuracy: %.6f' % tuple(score))
        print("--- Training time: %.2f seconds ---\n" % (time.time() - start_time))

    def evaluateModel(self):
        self._predictions = self._model.predict_classes(self._X_test)
        self._score = self._model.evaluate(self._X_test, self._y_test, batch_size = 315)
        self._y_test_target = np.array([x.argmax() for x in self._y_test])
        print('\nTest loss: %.6f, Test accuracy: %.6f' % tuple(self._score))

    def predictRandomDigit(self):
        index = np.random.randint(len(self._predictions))
        self.predictDigit(index)

    def predictDigit(self,index):
        if(index > len(self._y_test_target)-1):
            print(f'Index out of bound of data. Data size:{len(self._y_test_target)-1}')
        else:
            self._XTest = self._X_test[index].reshape(8, 8)
            plt.figure(figsize = (2, 2))
            plt.imshow(self._XTest, cmap = 'gray')
            plt.title(f'Pridiction: {self._predictions[index]}({self._y_test_target[index]})\n\n\n')
            plt.xticks([])
            plt.yticks([])
            plt.show()

    def runFullTest(self, epochNum,batch_size):
        self.prepareDataForMode()
        self.splitTrainingAndTestData()
        self.createModel()
        self.compileModel()
        self.trainModel(0,epochNum,batch_size)
        self.evaluateModel()
        # self.predictRandomDigit()

In [35]:
# Test KerasDemo01
kerasDemo=KerasDemo02()
# kerasDemo.printDataDescription()
# kerasDemo.plotRandomDigit()
# kerasDemo.plotDigit(50)
kerasDemo.prepareDataForMode()
kerasDemo.splitTrainingAndTestData()
kerasDemo.createModel()
kerasDemo.printModelSummary()
kerasDemo.compileModel()
# kerasDemo.trainModel()
kerasDemo.evaluateModel()
# kerasDemo.predictRandomDigit()
# kerasDemo.runFullTest(1)
for epochNum in range(1,30,5):
    kerasDemo.runFullTest(epochNum,100)

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_132 (Dense)            (None, 100)               6500      
_________________________________________________________________
dense_133 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_134 (Dense)            (None, 10)                510       
Total params: 12,060
Trainable params: 12,060
Non-trainable params: 0
_________________________________________________________________
None
2/2 [==============================] - 0s 1ms/step - loss: 7.4038 - accuracy: 0.1093

Test loss: 7.403821, Test accuracy: 0.109259
Fitting the model in 1 epochs
Model has been trained
4/4 [==============================] - 0s 798us/step - loss: 1.2764 - accuracy: 0.5823

Training loss: 1.276364, Training accuracy: 0.582339
--- Training time: 0.37 seconds ---

2/2 [=======